Creating the Provenance Data file

In [86]:
data_sources = ['CxG','Azimuth','HuBMAP','NeMo','Soumya','GTEx','Blue_Lake']

In [ ]:
import pandas as pd
import os
import re

provdata = pd.DataFrame()

for folder in os.listdir('csv'):
    if folder != ".DS_Store" and folder != "Skin_GE_Yingnan":
        data = folder
        data_source = None
        for source in data_sources:
            if re.search(source,folder,re.IGNORECASE):
                data_source = source
        provdata = provdata.append({'data': data, 'data source': data_source},ignore_index=True)

provdata['Dataset ID'] = provdata.apply(lambda row: os.listdir(os.path.join('csv',row['data'])),axis=1)

provdata = provdata.explode('Dataset ID')

In [88]:
def getdatasetname(file):
    data = pd.read_csv(file)
    if 'dataset_name' in data.columns:
        return data['dataset_name'][0]

provdata['Dataset Name'] = provdata.apply(lambda row: getdatasetname(os.path.join('csv',row['data'],row['Dataset ID'])),axis=1)

In [89]:
import requests

def get_doi_data(dataset_name: str):

    parameters = {'rows': 5, 'query.title': dataset_name}
    endpoint_url = f'https://api.crossref.org/works'
    response = requests.get(endpoint_url, params=parameters)
    try:
        response.raise_for_status()
        response_json = response.json()
    except Exception as e:
        print('Error : ', e)
        
    return response_json['message']['items'][0]['URL']

datalinks = pd.DataFrame()
datalinks['Dataset Name'] = provdata['Dataset Name'].unique()

datalinks['Link to Paper'] = datalinks.apply(lambda row: get_doi_data(row['Dataset Name']) if row['Dataset Name'] else None,axis=1)

In [90]:
provdata = provdata.merge(datalinks,how='left')

In [91]:
provdata.to_csv('Provenance Data.csv')

Total number of datasets

In [92]:
provdata.shape[0]

380

Number of CxG datasets

In [93]:
provdata.loc[provdata['data source'] == "CxG"].shape[0]

195

Number of unique papers for CxG datasets

In [94]:
provdata.loc[provdata['data source'] == "CxG","Link to Paper"].nunique()

49

Number of GTEx dataset organs

In [97]:
provdata.loc[provdata['data source'] == "GTEx","data"].nunique()

8

Number of CxG dataset organs

In [99]:
provdata.loc[provdata['data source'] == "CxG","data"].nunique()

49

Number of Tabula Sapiens organs 

In [100]:
import os

count = 0
for folder in os.listdir('csv'):
    if folder != ".DS_Store" and '53d208b0-2cfd-4366-9866-c3c6114081bc.csv' in os.listdir(os.path.join('csv',folder)):
        count += 1
print(count)

33
